# Private-Key Encryption

A **private-key encryption scheme** is defined by a **message space** $M$ and algorithms $\text{Gen}$, $\text{Enc}$ and $\text{Dec}$:

* $\text{Gen}$ (key generation algorithm): generates **key** $k$

* $\text{Enc}$ (encryption algorithm): takes key $k$ and **message** $m \in M$ as input; outputs **ciphertexst** $c$

$$c \gets \text{Enc}_k (m)$$

* $\text{Dec}$ (decryption algorithm): takes key $k$ and ciphertext $c$ as input; outputs message $m$ or "error"

$$m \coloneqq \text{Dec}_k (c)$$

with

$$\text{Dec}_k(\text{Enc}_k(m)) = m$$

#### Note on the Notation

The **left arrow** $\gets$ notation denotes assignment to the output of an algorithm that might be **randomized**. Meaning that the output of the algorithm may be different, even when run twice on the same set of inputs.

The **colon equals** $\coloneqq$ denotes an assignment to the output of a **deterministic** algorithm. 

So in this case, this means that we're allowing encryption to possibly be randomized, whereas we're assuming that decryption is deterministic.

## Examples

### Shift Cipher

### Vigenere Cipher